In [1]:
import numpy as np
import rasterio
from rasterio.mask import mask
import os

# from rasterio.features import bounds as feature_bounds
import pandas as pd
import geopandas as gpd

import func__adminstats_nested_polygons_splitheap as frr

In [2]:
# # load admin data. Its nees a specific structure.
# It needs a column called NUTS_ID, and a column cl_{x}_total for each category code x.

# TODO: move gpd.read_file(admin_GIS_path) into the pipeline. 
admin_GIS_path = 'in_data/admin/NUTS_branches_int.geojson'
polys = gpd.read_file(admin_GIS_path)

polys['cl_1_total'] = polys['cl_1_total'].fillna(-99)
polys['cl_2_total'] = polys['cl_2_total'].fillna(-99)
polys['cl_3_total'] = polys['cl_3_total'].fillna(-99)
polys['cl_4_total'] = polys['cl_4_total'].fillna(-99)

polys['cl_1_total'] = polys['cl_1_total'].astype(int)
polys['cl_2_total'] = polys['cl_2_total'].astype(int)
polys['cl_3_total'] = polys['cl_3_total'].astype(int)
polys['cl_4_total'] = polys['cl_4_total'].astype(int)

# define path for the urban areas. This is a fixed layer
urban_mask_path = 'in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/EU_urbanmask_1xx.tif'

fixed_layer_paths = [urban_mask_path]
fixed_layer_codes = [5]



foresttif_path         = "out_data/predictions/RF/EU_RF_mean_integer_compressed.tif"
grasslandtif_path      = "in_data/Corine_landcover/Grass_probability_final_layer_blured_Q2.tif"
# arab_croplandtif_path  = "in_data/Corine_landcover/ziga_filtered/arable_Crops_probability_final_layer_blured_Q2.tif"
# perm_croplandtif_path  = "in_data/Corine_landcover/ziga_filtered/permanent_Crops_probability_final_layer_blured_Q2.tif"

# I didn't have them EU wide, so I am using these for testing. 
arab_croplandtif_path  = "in_data/Corine_landcover/Crops_probability_final_layer_blured_mean.tif"
perm_croplandtif_path  = "in_data/Corine_landcover/Crops_probability_final_layer_blured_mean.tif"


probability_layers_paths = [
    foresttif_path,
    grasslandtif_path,
    arab_croplandtif_path,
    perm_croplandtif_path
]

probability_layers_codes = [1, 4, 2, 3]

region_raster_path  = 'in_data/admin/rasterized_NUTS_int.tif'



In [3]:
# polys = polys[polys['NUTS_ID'].isin( ['BE2', 'BE21', 'BE22', 'BE23', 'BE24', 'BE25'])]
# polys
# this does not work since 'BE' shows up in the tree, Im sure. 
# we can build in checks for that!!

In [4]:
config =  frr.ProcessingConfig(
    fixed_layer            = frr.LayerConfig(paths = fixed_layer_paths, codes = fixed_layer_codes),
    probability_layer      = frr.LayerConfig(paths = probability_layers_paths, codes = probability_layers_codes), 
    administrative_polygon = polys)

# once the data has been put into the containers, this line does the computation. 
# misc_value is given to the leftover pixels that we couldn't attribute to any of the admin classes.

# pipeline returns a numpy.ma.core.MaskedArray


In [ ]:
frr.pipeline(config, region_raster_path, misc_value = 10, output_folder = 'out_data/predictions/landcover_fillup/multilevel')

We split the whole tree into 248 subtrees.
215 of these subtrees are a single region.
The largest subtree has 13 (nested) regions
